# HTTPS Endpoint로 Model Serving

real-time과 batch predictions Endpoint를 모두 활용할 수 있으며, 이 HOL에서는 Real-time에 대해서만 설명합니다. 
사용 중이신 애플리케이션의 대기시간이 짧은 경우 Model은 real-time API로 배포하며 HTTPS를 통해 단일 예측 요청에 대해 빠르게 예측을 제공합니다.

In [1]:
import boto3
import sagemaker
import pandas as pd
import time

sess   = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

sm = boto3.Session().client(service_name='sagemaker', region_name=region)

### 이전 Notebook에서 Endpoints를 생성하신 경우에는 Endpoints를 삭제하신 후 이 노트북을 생성하시기 바랍니다. 그렇지 않은 경우 HOL를 수행하시는 동안에 ResourceLimitExceeded error를 보실 수 있습니다.


In [2]:
list_endpoints = sm.list_endpoints()

for ep in list_endpoints['Endpoints']:
    sm.delete_endpoint(EndpointName=ep['EndpointName'])
    

NextToken = 'None'
while NextToken !='':
    lec = sm.list_endpoint_configs(NextToken=NextToken) if NextToken != 'None' else sm.list_endpoint_configs()
    for epc in lec['EndpointConfigs']:
        print(epc['EndpointConfigName'])
        sm.delete_endpoint_config(EndpointConfigName=epc['EndpointConfigName'])
        time.sleep(3)
    NextToken = lec['NextToken'] if lec.get('NextToken') else ''

NextToken = 'None'
while NextToken !='':
    lec = sm.list_models(NextToken=NextToken) if NextToken != 'None' else sm.list_models()
    for epc in lec['Models']:
        print(epc['ModelName'])
        sm.delete_model(ModelName=epc['ModelName'])
        time.sleep(3)
    NextToken = lec['NextToken'] if lec.get('NextToken') else ''

automl-dm-epc-15-05-41-41
endpoint-pet-classifier-1599199760
automl-dm-model-15-05-41-41
automl-dm-15-05-41-41-automl-dm--dpp0-model-1dfbbd0e5f8e464689f
automl-dm-15-05-41-41-automl-dm--dpp1-model-8a74914373eb407d8a5
automl-dm-15-05-41-41-automl-dm--dpp2-model-ccb158e1f126483d82a
pytorch-inference-eia-2020-09-04-06-09-29-156


In [3]:
%store -r

In [4]:
print(training_job_name)

tensorflow-training-2020-09-15-06-48-39-100


# Notebook 내 Model 복사

In [5]:
!aws s3 cp s3://$bucket/$training_job_name/output/model.tar.gz ./model.tar.gz

download: s3://sagemaker-us-east-1-322537213286/tensorflow-training-2020-09-15-06-48-39-100/output/model.tar.gz to ./model.tar.gz


In [6]:
!tar -xvzf ./model.tar.gz

tensorboard/
metrics/
metrics/confusion_matrix.png
transformers/
transformers/fine-tuned/
transformers/fine-tuned/tf_model.h5
transformers/fine-tuned/config.json
tensorflow/
tensorflow/saved_model/
tensorflow/saved_model/0/
tensorflow/saved_model/0/saved_model.pb
tensorflow/saved_model/0/variables/
tensorflow/saved_model/0/variables/variables.data-00000-of-00002
tensorflow/saved_model/0/variables/variables.data-00001-of-00002
tensorflow/saved_model/0/variables/variables.index
tensorflow/saved_model/0/assets/


In [7]:
!saved_model_cli show --all --dir ./tensorflow/saved_model/0/

2020-09-15 07:40:54.269855: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libnvinfer.so.6'; dlerror: libnvinfer.so.6: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-10.0/lib64:/usr/local/cuda-10.0/extras/CUPTI/lib64:/usr/local/cuda-10.0/lib:/usr/local/cuda-10.0/efa/lib:/opt/amazon/efa/lib:/opt/amazon/efa/lib64:/usr/lib64/openmpi/lib/:/usr/local/lib:/usr/lib:/usr/local/mpi/lib:/lib/:/usr/lib64/openmpi/lib/:/usr/local/lib:/usr/lib:/usr/local/mpi/lib:/lib/:/usr/lib64/openmpi/lib/:/usr/local/lib:/usr/lib:/usr/local/mpi/lib:/lib/:
2020-09-15 07:40:54.269949: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libnvinfer_plugin.so.6'; dlerror: libnvinfer_plugin.so.6: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-10.0/lib64:/usr/local/cuda-10.0/extras/CUPTI/lib64:/usr/local/cuda-10.0/lib:/usr/local/cuda-10.0/ef

# Model 배포

단일 Model에 대해 기본적으로 `EndpointConfig`를 생성합니다. 다음 노트북에서 canary 배포와 A/B testing을 지원하는 `EndpointConfig` 전략을 수행할 계획입니다. 

_참고: 리전에 따라 기본적으로 제공되는 다양한 버전/Deep leanring 프레임워크 별로 deep learning 이미지를 제공하고 있으며, 필요에 따라 해당 image를 받아서 추가해서 custom 이미지를 생성할 수 있습니다._

https://docs.aws.amazon.com/deep-learning-containers/latest/devguide/deep-learning-containers-images.html


또한, SageMaker에서 학습한 모델이 아닌 경우에도 XXXXX.tar.gz로 압축하여 예측에 활용할 수 있습니다.

In [8]:
from sagemaker.tensorflow.serving import Model

model = Model(model_data='s3://{}/{}/output/model.tar.gz'.format(bucket, training_job_name),
              role=role,
              framework_version='2.0.0') # Elastic Inference does not yet support TF 2.1.0 as of sagemaker==1.56.1

Parameter image will be renamed to image_uri in SageMaker Python SDK v2.


In [9]:
deployed_model = model.deploy(initial_instance_count=1, # Should use >=2 for high(er) availability 
                              instance_type='ml.m5.large',
#                              accelerator_type='ml.eia2.medium',
                              wait=False)

endpoint_name = deployed_model.endpoint

print('Endpoint name:  {}'.format(endpoint_name))

'create_image_uri' will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.


Endpoint name:  tensorflow-inference-2020-09-15-07-41-00-789


# Experiment 내 배포 추적

In [10]:
print(experiment_name)

Amazon-Customer-Reviews-BERT-Experiment-1600152510


In [11]:
print(trial_name)

trial-1600152510


In [12]:
import time
from smexperiments.trial import Trial

timestamp = '{}'.format(int(time.time()))

trial = Trial.load(trial_name=trial_name)
print(trial)

Trial(sagemaker_boto_client=<botocore.client.SageMaker object at 0x7fa8bd486c50>,trial_name='trial-1600152510',trial_arn='arn:aws:sagemaker:us-east-1:322537213286:experiment-trial/trial-1600152510',display_name='trial-1600152510',experiment_name='Amazon-Customer-Reviews-BERT-Experiment-1600152510',creation_time=datetime.datetime(2020, 9, 15, 6, 48, 30, 767000, tzinfo=tzlocal()),created_by={},last_modified_time=datetime.datetime(2020, 9, 15, 7, 40, 3, 47000, tzinfo=tzlocal()),last_modified_by={},response_metadata={'RequestId': '157e57c4-d0e2-48a7-9963-5ded140ddc46', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '157e57c4-d0e2-48a7-9963-5ded140ddc46', 'content-type': 'application/x-amz-json-1.1', 'content-length': '326', 'date': 'Tue, 15 Sep 2020 07:41:02 GMT'}, 'RetryAttempts': 0})


In [13]:
from smexperiments.tracker import Tracker

tracker_deploy = Tracker.create(display_name='deploy', 
                                sagemaker_boto_client=sm)

deploy_trial_component_name = tracker_deploy.trial_component.trial_component_name
print('Deploy trial component name {}'.format(deploy_trial_component_name))

Deploy trial component name TrialComponent-2020-09-15-074102-uzmt


### Trial에 Component로 `deploy` Trial Component 와 Tracker를 추가합니다.

In [14]:
trial.add_trial_component(tracker_deploy.trial_component)

# Endpoint Name 추적

In [15]:
tracker_deploy.log_parameters({
    'endpoint_name': endpoint_name,
})

# must save after logging
tracker_deploy.trial_component.save()

TrialComponent(sagemaker_boto_client=<botocore.client.SageMaker object at 0x7fa8bcae2400>,trial_component_name='TrialComponent-2020-09-15-074102-uzmt',display_name='deploy',trial_component_arn='arn:aws:sagemaker:us-east-1:322537213286:experiment-trial-component/trialcomponent-2020-09-15-074102-uzmt',response_metadata={'RequestId': '9fa8c1cc-848f-4b08-8266-58d39d8f5124', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '9fa8c1cc-848f-4b08-8266-58d39d8f5124', 'content-type': 'application/x-amz-json-1.1', 'content-length': '129', 'date': 'Tue, 15 Sep 2020 07:41:03 GMT'}, 'RetryAttempts': 0},parameters={'endpoint_name': 'tensorflow-inference-2020-09-15-07-41-00-789'},input_artifacts={},output_artifacts={})

In [16]:
from IPython.core.display import display, HTML

display(HTML('<b>Review <a href="https://console.aws.amazon.com/sagemaker/home?region={}#/endpoints/{}">REST Endpoint</a></b>'.format(region, endpoint_name)))


In [17]:
client = boto3.client('sagemaker')
waiter = client.get_waiter('endpoint_in_service')
waiter.wait(EndpointName=endpoint_name)

<h2><span style="color:red">위 Endpoint가 Deploy되기 전까지 기다려 주시기 바랍니다.</span></h2>

In [18]:
from sagemaker.analytics import ExperimentAnalytics

lineage_table = ExperimentAnalytics(
    sagemaker_session=sess,
    experiment_name=experiment_name,
    metric_names=['validation:accuracy'],
    sort_by="CreationTime",
    sort_order="Ascending",
)

lineage_df = lineage_table.dataframe()
lineage_df.shape

(4, 57)

In [19]:
lineage_df

,TrialComponentName,DisplayName,max_seq_length,test_split_percentage,train_split_percentage,validation_split_percentage,raw_data_s3_uri - MediaType,raw_data_s3_uri - Value,SourceArn,SageMaker.ImageUri,...,train - Value,validation - MediaType,validation - Value,SageMaker.Checkpoints - MediaType,SageMaker.Checkpoints - Value,SageMaker.DebugHookOutput - MediaType,SageMaker.DebugHookOutput - Value,SageMaker.ModelArtifact - MediaType,SageMaker.ModelArtifact - Value,endpoint_name
0,TrialComponent-2020-09-15-064830-dysr,prepare,128.0,0.05,0.9,0.05,s3/uri,s3://sagemaker-us-east-1-322537213286/amazon-r...,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,tensorflow-training-2020-09-15-06-48-39-100-aw...,train,128.0,NaN,NaN,NaN,NaN,NaN,arn:aws:sagemaker:us-east-1:322537213286:train...,763104351884.dkr.ecr.us-east-1.amazonaws.com/t...,...,s3://sagemaker-us-east-1-322537213286/sagemake...,NaN,s3://sagemaker-us-east-1-322537213286/sagemake...,NaN,s3://sagemaker-us-east-1-322537213286/checkpoi...,NaN,s3://sagemaker-us-east-1-322537213286/,NaN,s3://sagemaker-us-east-1-322537213286/tensorfl...,NaN
2,tensorflow-training-2020-09-15-07-39-58-824-aw...,train,128.0,NaN,NaN,NaN,NaN,NaN,arn:aws:sagemaker:us-east-1:322537213286:train...,763104351884.dkr.ecr.us-east-1.amazonaws.com/t...,...,s3://sagemaker-us-east-1-322537213286/sagemake...,NaN,s3://sagemaker-us-east-1-322537213286/sagemake...,NaN,s3://sagemaker-us-east-1-322537213286/checkpoi...,NaN,s3://sagemaker-us-east-1-322537213286/,NaN,NaN,NaN
3,TrialComponent-2020-09-15-074102-uzmt,deploy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,tensorflow-inference-2020-09-15-07-41-00-789


# Prediction 수행

###  Raw Text를 BERT Tokens로 변환하기 위한 Request Handler 설정

In [20]:
class RequestHandler(object):
    import json
    
    def __init__(self, tokenizer, max_seq_length):
        self.tokenizer = tokenizer
        self.max_seq_length = max_seq_length

    def __call__(self, instances):
        transformed_instances = []

        for instance in instances:
            encode_plus_tokens = tokenizer.encode_plus(instance,
                                                       pad_to_max_length=True,
                                                       max_length=self.max_seq_length)

            input_ids = encode_plus_tokens['input_ids']
            input_mask = encode_plus_tokens['attention_mask']
            segment_ids = [0] * self.max_seq_length

            transformed_instance = {"input_ids": input_ids, 
                                    "input_mask": input_mask, 
                                    "segment_ids": segment_ids}

            transformed_instances.append(transformed_instance)

        transformed_data = {"instances": transformed_instances}

        return json.dumps(transformed_data)

###  BERT Response를 Predicted Classes로 변환하기 위한 Response Handler 설정

In [21]:
class ResponseHandler(object):
    import json
    import tensorflow as tf
    
    def __init__(self, classes):
        self.classes = classes
    
    def __call__(self, response, accept_header):
        import tensorflow as tf

        response_body = response.read().decode('utf-8')

        response_json = json.loads(response_body)

        log_probabilities = response_json["predictions"]

        predicted_classes = []

        # Convert log_probabilities => softmax (all probabilities add up to 1) => argmax (final prediction)
        for log_probability in log_probabilities:
            softmax = tf.nn.softmax(log_probability)    
            predicted_class_idx = tf.argmax(softmax, axis=-1, output_type=tf.int32)
            predicted_class = self.classes[predicted_class_idx]
            predicted_classes.append(predicted_class)

        return predicted_classes

In [22]:
import json
from sagemaker.tensorflow.serving import Predictor
from transformers import DistilBertTokenizer

tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

request_handler = RequestHandler(tokenizer=tokenizer,
                                 max_seq_length=128)

response_handler = ResponseHandler(classes=[1, 2, 3, 4, 5])

predictor = Predictor(endpoint_name=endpoint_name,
                      sagemaker_session=sess,
                      serializer=request_handler,
                      deserializer=response_handler,
                      content_type='application/json',
                      model_name='saved_model',
                      model_version=0)

INFO:transformers.file_utils:TensorFlow version 2.1.0 available.
INFO:transformers.tokenization_utils:loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at /home/ec2-user/.cache/torch/transformers/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084


In [23]:
import tensorflow as tf
import json
    
reviews = ["This is great!", 
           "This is not good."]

predicted_classes = predictor.predict(reviews)

for predicted_class, review in zip(predicted_classes, reviews):
    print('[Predicted Star Rating: {}]'.format(predicted_class), review)

[Predicted Star Rating: 5] This is great!
[Predicted Star Rating: 5] This is not good.


In [24]:
%store endpoint_name

Stored 'endpoint_name' (str)


# Using API Gateway with SageMaker Endpoints

https://aws.amazon.com/blogs/machine-learning/creating-a-machine-learning-powered-rest-api-with-amazon-api-gateway-mapping-templates-and-amazon-sagemaker/

# Delete Endpoint

In [25]:
# client = boto3.client('sagemaker')

# client.delete_endpoint(
#     EndpointName=endpoint_name
# )

{'ResponseMetadata': {'RequestId': 'fff98548-7eaa-4e0c-9b44-4ca9c5a66e4e',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'fff98548-7eaa-4e0c-9b44-4ca9c5a66e4e',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '0',
   'date': 'Tue, 15 Sep 2020 07:48:42 GMT'},
  'RetryAttempts': 0}}